# Recommend posts for the given user
#####(Collaborative Filtering)

In [1]:
#LIBRARIES

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
users_df = pd.read_csv("/content/users.csv")
views_df = pd.read_csv("/content/views.csv")
posts_df = pd.read_csv("/content/posts.csv")

In [3]:
users_df = users_df.rename(columns={'_id':'user_id'})

In [4]:
users_df.head()

,user_id,name,gender,academics
0,5d60098a653a331687083238,Nivesh Singh Chauhan,male,undergraduate
1,5d610ae1653a331687083239,Gaurav Sharma,male,graduate
2,5d618359fc5fcf3bdd9a0910,Akshay Mishra,male,undergraduate
3,5d6d2bb87fa40e1417a49315,Saksham Mathur,male,undergraduate
4,5d7c994d5720533e15c3b1e9,Varun Chowhan,male,undergraduate


In [5]:
posts_df = posts_df.rename(columns={'_id':'post_id'})
posts_df['category'] = posts_df['category'].str.replace('|',' ')

In [6]:
posts_df.head()

,post_id,title,category,post_type
0,5d62abaa65218653a132c956,hello there,Plant Biotechnology,blog
1,5d6d39567fa40e1417a4931c,Ml and AI,Artificial Intelligence Machine Learning Infor...,blog
2,5d7d23315720533e15c3b1ee,What is an Operating System ?,Operating Systems,blog
3,5d7d405e5720533e15c3b1f3,Lord Shiva,Drawings,artwork
4,5d80dfbc6c53455f896e600e,How Competition law evolved?,Competition Laws,blog


In [7]:
views_df.head()

,user_id,post_id,timestamp
0,5df49b32cc709107827fb3c7,5ec821ddec493f4a2655889e,2020-06-01T10:46:45.131Z
1,5ed3748576027d35905ccaab,5ed4cbadbd514d602c1531a6,2020-06-01T09:39:20.021Z
2,5ed0defa76027d35905cc2de,5eac305f10426255a7aa9dd3,2020-06-01T08:12:42.682Z
3,5ed0defa76027d35905cc2de,5ed1ff0276027d35905cc60d,2020-06-01T08:10:23.880Z
4,5ed0defa76027d35905cc2de,5ed3820f76027d35905ccac8,2020-06-01T08:08:54.124Z


Meta Data 

In [8]:
df_1 = pd.merge(users_df, views_df, on ='user_id')

In [9]:
df = pd.merge(df_1, posts_df, on ='post_id')

In [10]:
df.head()

,user_id,name,gender,academics,post_id,timestamp,title,category,post_type
0,5d60098a653a331687083238,Nivesh Singh Chauhan,male,undergraduate,5ed3820f76027d35905ccac8,2020-05-31T18:01:54.308Z,Configure Docker with Django; PostgreSQL; Pg-a...,Computer Technology Computer Application Infor...,blog
1,5ecb979eeaff6b0c3a58a4f0,Kanika Sharma,female,undergraduate,5ed3820f76027d35905ccac8,2020-05-31T20:40:18.693Z,Configure Docker with Django; PostgreSQL; Pg-a...,Computer Technology Computer Application Infor...,blog
2,5ed0defa76027d35905cc2de,Asif Hossain,male,undergraduate,5ed3820f76027d35905ccac8,2020-06-01T08:08:54.124Z,Configure Docker with Django; PostgreSQL; Pg-a...,Computer Technology Computer Application Infor...,blog
3,5ed3748576027d35905ccaab,Parth Vijay,male,undergraduate,5ed3820f76027d35905ccac8,2020-05-31T10:08:37.079Z,Configure Docker with Django; PostgreSQL; Pg-a...,Computer Technology Computer Application Infor...,blog
4,5d60098a653a331687083238,Nivesh Singh Chauhan,male,undergraduate,5ed1ff0276027d35905cc60d,2020-05-31T08:21:29.911Z,AWS services and how to launch OS on AWS Cloud,Computer Technology Cloud Computing,blog


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1447 entries, 0 to 1446
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     1447 non-null   object
 1   name        1447 non-null   object
 2   gender      1447 non-null   object
 3   academics   1447 non-null   object
 4   post_id     1447 non-null   object
 5   timestamp   1447 non-null   object
 6   title       1447 non-null   object
 7   category    1447 non-null   object
 8    post_type  1447 non-null   object
dtypes: object(9)
memory usage: 113.0+ KB


In [12]:
df.isnull().sum()

user_id       0
name          0
gender        0
academics     0
post_id       0
timestamp     0
title         0
category      0
 post_type    0
dtype: int64

There were few empty cells in the raw data under the category column, but since it is a small dataset, I manually handeled the empty cells, I saw all the empty cells were projects of various types. So, I assigned them a category 'Project'

In [13]:
df['category'].value_counts()

Photography                                                                           254
Drawings                                                                              149
Project                                                                                75
Painting                                                                               75
Computer Technology Machine Learning                                                   40
                                                                                     ... 
Biotechnology Environmental Biotechnology                                               1
Computer Technology Hardware Information Technology                                     1
Social Work Social Tech                                                                 1
Human Resources Performance In Organization Organizational Behaviour HR Management      1
Mass Media International Relations                                                      1
Name: cate

In [14]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label = le.fit_transform(df['category'])
l = le.fit_transform(df['post_id'])
a = le.fit_transform(df['timestamp'])

In [15]:
df['metadata_1'] = label
df['metadata_2'] = a
df['metadata_3'] = l
df['metadata'] = df['metadata_1'].map(str)+df['metadata_2'].map(str)+df['metadata_3'].map(str)
df['metadata'] = df["metadata"].astype(str).astype(int)
df = df.drop(['metadata_1', 'metadata_2', 'metadata_3'], axis = 'columns')

In [16]:
df.head()

,user_id,name,gender,academics,post_id,timestamp,title,category,post_type,metadata
0,5d60098a653a331687083238,Nivesh Singh Chauhan,male,undergraduate,5ed3820f76027d35905ccac8,2020-05-31T18:01:54.308Z,Configure Docker with Django; PostgreSQL; Pg-a...,Computer Technology Computer Application Infor...,blog,471430489
1,5ecb979eeaff6b0c3a58a4f0,Kanika Sharma,female,undergraduate,5ed3820f76027d35905ccac8,2020-05-31T20:40:18.693Z,Configure Docker with Django; PostgreSQL; Pg-a...,Computer Technology Computer Application Infor...,blog,471435489
2,5ed0defa76027d35905cc2de,Asif Hossain,male,undergraduate,5ed3820f76027d35905ccac8,2020-06-01T08:08:54.124Z,Configure Docker with Django; PostgreSQL; Pg-a...,Computer Technology Computer Application Infor...,blog,471442489
3,5ed3748576027d35905ccaab,Parth Vijay,male,undergraduate,5ed3820f76027d35905ccac8,2020-05-31T10:08:37.079Z,Configure Docker with Django; PostgreSQL; Pg-a...,Computer Technology Computer Application Infor...,blog,471418489
4,5d60098a653a331687083238,Nivesh Singh Chauhan,male,undergraduate,5ed1ff0276027d35905cc60d,2020-05-31T08:21:29.911Z,AWS services and how to launch OS on AWS Cloud,Computer Technology Cloud Computing,blog,401414470


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1447 entries, 0 to 1446
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     1447 non-null   object
 1   name        1447 non-null   object
 2   gender      1447 non-null   object
 3   academics   1447 non-null   object
 4   post_id     1447 non-null   object
 5   timestamp   1447 non-null   object
 6   title       1447 non-null   object
 7   category    1447 non-null   object
 8    post_type  1447 non-null   object
 9   metadata    1447 non-null   int64 
dtypes: int64(1), object(9)
memory usage: 124.4+ KB


## Pivot Matrix 

In [18]:
post_features_df = df.pivot_table(index = 'name', columns = 'title', values='metadata').fillna(0)

In [19]:
post_features_df.head()


title,'The Virtual ME',10 Best Artificial Intelligence (AI) Courses Online for 2020,10 Best UI UX Design Course & Certification [2020],100 Free University Courses Online [2020],20 Best Data Science Certification & Courses for 2020,21 day Lock down. Quarantine,3D composition.,5 Best Computer Networking Courses & Certification [2020],5 Best Productivity Web Apps For Students.,5 points to consider while choosing a pathology lab,6 Best + Free PLC Programming Training & Course [2020 UPDATED],7 Best Python Data Science Courses & Certification [2020],7 Steps To Prevent COVID-19,7 Steps To Stay Safe From Corona Virus,8 Best Machine Learning Courses for 2020,8 Free Data Science Courses & Tutorials [2020],A solar-powered microbrewery has design-led energy in Sweden,A stone bridge on the full moon night,AUDIT REMARK,AWS CLI Setup in Mac,AWS services and how to launch OS on AWS Cloud,Abacus,Acceptance!,Accounting As A Financial Institution.,Advantages of Online Shopping Over Offline Shopping,Aesthetic,Aesthetic.,Aesthetics,Alone,Always Remember !!!,Always do Right 👍🏻,Analysis Of Successful Companies For the Past 5 years,App Development,Applications of mobile computing,Are We Alone In The Universe?,Armstrong No.(C# .Net),Art,Art Expo 2020,Artificial Intelligence,Avengers Theme - Acoustic guitar,...,What is an Operating System ?,What sports will look like in the future,Why Do Millennials Change Jobs?,Wildlife Creation.,Will AI replace humans intellectual work?,Will IPL happen this year,WisdomLeaf,Women power,Women: goddess,Work From Home: Working Under Corona's Effect,Workforce Diversity as a Challenge,Xbox Series X To Launch With 'Thousands' of Games,You are late.,You are one in a kind!,You know if you've been here.,You; are enough!,Your reaction can change your destination.,Zero-Waste Lifestyle,aloof,based on your imagination,deep thoughts,early mornings with Gang,engrossed,filtered,freeze,gripping,hello there,innocence,insect,monument,peace,portraying goddess,screw town,screw2,silence,street art; Jaisalmer,trekking,upside down,आमची मुंबाई !!!,🍉 Watermelon lemon cooler
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Aanjan Prasath G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,164287205.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Aayushree Khokhar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,164804348.0,164414297.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,164315242.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Abhinav Mishra,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Abhishank Gupta,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,71490322.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Abhishek Kumar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1546052.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Building Model

In [20]:
from scipy.sparse import csr_matrix

In [21]:
post_features_df_matrix = csr_matrix(post_features_df.values)
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(post_features_df_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

I have choosen cosine metric because even after trying other metrices such as 'euclidean', 'minkoswki', cosine was getting better result. Also, while building a recommendation system with collaborative filtering, cosine metric is considered the best.

In [22]:
post_features_df.shape

(118, 477)

In [23]:
query_index = np.random.choice(post_features_df.shape[0])
print(query_index)
distances , indices = model_knn.kneighbors(post_features_df.iloc[query_index,:].values.reshape(1,-1), n_neighbors=6)

61


In [24]:
for i in range(0, len(distances.flatten())):
  if i == 0:
    print("recommendation for {0}:\n".format(post_features_df.index[query_index]))
    post_features_df = df.pivot_table(index = 'title', columns = 'name', values='metadata').fillna(0)
    post_features_df_matrix = csr_matrix(post_features_df.values)
    print("(Due To The Post) {0}:\n".format(post_features_df.index[query_index]))
  else:
    post_features_df = df.pivot_table(index = 'title', columns = 'name', values='metadata').fillna(0)
    post_features_df_matrix = csr_matrix(post_features_df.values)
    print('{0}: {1}, with distance of {2}:'.format(i, post_features_df.index[indices.flatten()[i]], distances.flatten()[i]))
    post_features_df = df.pivot_table(index = 'name', columns = 'title', values='metadata').fillna(0)
    post_features_df_matrix = csr_matrix(post_features_df.values)

recommendation for Nihal Singh Shekhawat:

(Due To The Post) Bloom🌸:

1: Abacus, with distance of 0.9491985308387677:
2: 6 Best + Free PLC Programming Training & Course [2020 UPDATED], with distance of 0.9961793420312776:
3: Closeness to nature, with distance of 1.0:
4: Childhood Sketch, with distance of 1.0:
5: Change., with distance of 1.0:


# Recommend similar posts for the given post
####(Content Filtering)

In [25]:
import pandas as pd
import numpy as np
import seaborn as sns


In [26]:
df = pd.read_csv("/content/posts.csv")

In [27]:
df.head()

,_id,title,category,post_type
0,5d62abaa65218653a132c956,hello there,Plant Biotechnology,blog
1,5d6d39567fa40e1417a4931c,Ml and AI,Artificial Intelligence|Machine Learning|Infor...,blog
2,5d7d23315720533e15c3b1ee,What is an Operating System ?,Operating Systems,blog
3,5d7d405e5720533e15c3b1f3,Lord Shiva,Drawings,artwork
4,5d80dfbc6c53455f896e600e,How Competition law evolved?,Competition Laws,blog


In [28]:
df.isnull().sum()

_id           0
title         0
category      0
 post_type    0
dtype: int64

In [29]:
df.category.value_counts()

Photography                                                              83
Drawings                                                                 47
Project                                                                  26
Painting                                                                 20
Visual Arts                                                              10
                                                                         ..
Legal Studies|Company Law|Legal System|Banking Law                        1
Professionalism                                                           1
Computer Technology|Computer Application|Operating Systems                1
Photography|Fashion Design|Visual Arts|Graphic Design|Artistic design     1
Accounting|Financial Accounting                                           1
Name: category, Length: 232, dtype: int64

In [30]:
df.head(1)['category']

0    Plant Biotechnology
Name: category, dtype: object

Category column contains all the categorical inputs. To use it for recommending system we can convert it into a sparse matrix using the NLP concept Tfidf.

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvf = TfidfVectorizer(min_df = 3, max_features=None,
                      strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}',
                      ngram_range=(1,5),
                      stop_words = 'english')

## Prediction Based on Category

In [32]:
df['category'] = df['category'].fillna('')

In [33]:
tvf_matrix = tvf.fit_transform(df['category'])

In [34]:
tvf_matrix

<493x252 sparse matrix of type '<class 'numpy.float64'>'
	with 1881 stored elements in Compressed Sparse Row format>

In [35]:
tvf_matrix.shape

(493, 252)

In [36]:
from sklearn.metrics.pairwise import sigmoid_kernel

sig = sigmoid_kernel(tvf_matrix, tvf_matrix)

In [37]:
indices = pd.Series(df.index, index=df['title'])

In [38]:
indices

title
hello there                                                              0
Ml and AI                                                                1
What is an Operating System ?                                            2
Lord Shiva                                                               3
How Competition law evolved?                                             4
                                                                      ... 
CI/CD using GitHub Actions                                             488
Configure Docker with Django; PostgreSQL; Pg-admin & Elasticsearch     489
Recommendation Engine                                                  490
Face Recognition using Transfer Learning.                              491
Configure CI/CD Pipeline in GitLab and deployment to server via SSH    492
Length: 493, dtype: int64

In [39]:
def give_rec(title, sig=sig):

  idx = indices[title]
  sig_scores = list(enumerate(sig[idx]))
  sig_scores = sorted(sig_scores, key = lambda x: x[1], reverse = True)
  sig_scores = sig_scores[1:6]
  post_indices = [i[0] for i in sig_scores]
  return df['title'].iloc[post_indices]

In [40]:
give_rec('Ml and AI')

375    6 Best + Free PLC Programming Training & Cours...
169             What sports will look like in the future
122                              Artificial Intelligence
170                                         Types Of AI.
291             8 Best Machine Learning Courses for 2020
Name: title, dtype: object